* 目的：日本語学習済みBERTを導入

* 動作環境：Google Colaboratory

# 日本語学習済みBERTのインストール

①以下のURLから"通常版:Japanese_L-12_H-768_A-12_E-30_BPE.zip (1.6G)"をダウンロード <br>
http://nlp.ist.i.kyoto-u.ac.jp/index.php?BERT%E6%97%A5%E6%9C%AC%E8%AA%9EPretrained%E3%83%A2%E3%83%87%E3%83%AB#k1aa6ee3 <br>
②ダウンロードしたものをGoogleのマイドライブの好きな場所に保存しておく <br>

In [ ]:
#ドライブにマウント
from google.colab import drive
drive.mount('/content/drive')

# JUMAN++のインストール
* BERTを動作するのに必要

In [ ]:
%cd drive/My Drive/"JUMAN++の保存場所"

In [ ]:
!wget "http://lotus.kuee.kyoto-u.ac.jp/nl-resource/jumanpp/jumanpp-1.02.tar.xz"

In [ ]:
!tar xJvf jumanpp-1.02.tar.xz

In [ ]:
cd jumanpp-1.02

In [ ]:
!./configure

In [ ]:
!make

In [ ]:
!sudo make install

# pyknpのインストール
* 同様にBERT動作のために必要

In [ ]:
!sudo pip install pyknp

# JUMAN++に対応させるためにコードの一部を改修

* Colabの左側にあるファイルから /"JUMAN++の保存場所"/juman++/jumanpp-1.02/bert/tokenization.py を開く
* tokenization.pyの真ん中やや上を改修する

In [ ]:
class FullTokenizer(object):
  """Runs end-to-end tokenziation."""

  def __init__(self, vocab_file, do_lower_case=True):
    self.vocab = load_vocab(vocab_file)
    self.inv_vocab = {v: k for k, v in self.vocab.items()}

    # JUMAN++を用いた日本語対応の場合、BasicTokenizerを使わない。
    # self.basic_tokenizer = BasicTokenizer(do_lower_case=do_lower_case)
    self.jumanpp_tokenizer = JumanPPTokenizer()
    self.wordpiece_tokenizer = WordpieceTokenizer(vocab=self.vocab)

  def tokenize(self, text):
    split_tokens = []
    # JUMAN++を用いた日本語対応用。
    # 設定で切り替えることが出来ればよいのだが。
    #for token in self.basic_tokenizer.tokenize(text):
    for token in self.jumanpp_tokenizer.tokenize(text):
      for sub_token in self.wordpiece_tokenizer.tokenize(token):
        split_tokens.append(sub_token)

    return split_tokens

  def convert_tokens_to_ids(self, tokens):
    return convert_by_vocab(self.vocab, tokens)

  def convert_ids_to_tokens(self, ids):
    return convert_by_vocab(self.inv_vocab, ids)

* JumanPPTokenizerの追加
* 同様のファイル内の一番下に以下のコードを追加

In [ ]:
class JumanPPTokenizer(BasicTokenizer):
  def __init__(self):
    """
        日本語専用トークナイザの構築。
        JUMAN++ を使用する。
    """
    from pyknp import Juman

    self.do_lower_case = False
    self._jumanpp = Juman()

  def tokenize(self, text):
    """Tokenizes a piece of text."""
    text = convert_to_unicode(text.replace(' ', ''))
    text = self._clean_text(text)

    juman_result = self._jumanpp.analysis(text)
    split_tokens = []
    for mrph in juman_result.mrph_list():
      split_tokens.extend(self._run_split_on_punc(mrph.midasi))

    output_tokens = whitespace_tokenize(" ".join(split_tokens))
    print(split_tokens)
    return output_tokens

# 参考URL
* https://qiita.com/takahashi_yukou/items/b81319b8ef6cee13cb1b
* https://dev.classmethod.jp/machine-learning/bert-text-embedding/